In [ ]:
import requests
import json
import os

class Agent:
    def __init__(self, name, instructions, model, handoffs=None):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.handoffs = handoffs or []
        self.ai_proxy_url = "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"
        self.ai_proxy_token = os.getenv("AIPROXY_TOKEN")  # Replace with your actual token
    
    def process(self, user_input):
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.ai_proxy_token}"
        }
        
        # Note: AI Proxy only supports gpt-4o-mini
        if self.model != "gpt-4o-mini":
            print(f"Warning: Model {self.model} not supported. Using gpt-4o-mini instead.")
        
        data = {
            "model": "gpt-4o-mini",
            "messages": [
                {"role": "system", "content": self.instructions},
                {"role": "user", "content": user_input}
            ]
        }
        
        response = requests.post(self.ai_proxy_url, headers=headers, json=data)
        
        # Check for headers with usage info
        cost = response.headers.get("cost", "N/A")
        monthly_cost = response.headers.get("monthlyCost", "N/A")
        monthly_requests = response.headers.get("monthlyRequests", "N/A")
        
        print(f"Request cost: ${cost}")
        print(f"Monthly usage: ${monthly_cost} ({monthly_requests} requests)")
        
        if response.status_code == 200:
            result = response.json()
            return result["choices"][0]["message"]["content"]
        else:
            return f"Error: {response.status_code} - {response.text}"
    
    def route(self, user_input):
        if not self.handoffs:
            return self.process(user_input)
        
        # For triage agent, first determine which agent to route to
        routing_decision = self.process(f"Route this request to the appropriate agent: {user_input}")
        
        # Simple parsing of the triage response to determine which agent to use
        for agent in self.handoffs:
            if agent.name.lower() in routing_decision.lower():
                print(f"Routing to {agent.name}")
                return agent.process(user_input)
        
        # Default handling if no agent is identified
        print("No specific agent identified, handling with triage agent")
        return self.process(user_input)

# Initialize your agents
booking_agent = Agent(
    name="Booking Agent", 
    instructions="Book tickets for a specific city and date", 
    model="gpt-4o-mini"
)

refund_agent = Agent(
    name="Refund Agent", 
    instructions="Process refunds for tickets booked", 
    model="gpt-4o-mini"
)

triage_agent = Agent(
    name="Triage Agent", 
    instructions="Triage the request and route it to the appropriate agent", 
    model="gpt-4o-mini", 
    handoffs=[booking_agent, refund_agent]
)

# Example usage
if __name__ == "__main__":
    user_query = input("How can I help you today? ")
    response = triage_agent.route(user_query)
    print("\nResponse:", response)

In [ ]:
import requests
import json
import time
import datetime
import os
from dotenv import load_dotenv
load_dotenv()
from typing import List, Dict, Any, Optional, Union

class LLMClient:
    def __init__(self, model = "gpt-4o-mini"):
        self.model = model
        self.ai_proxy_url = "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"
        self.ai_proxy_token = os.getenv("AIPROXY_TOKEN", os.getenv("AIPROXY_TOKEN"))  # Replace with your actual token
        self.conversation_history = []
        self.log_file = "llm_interaction_logs.json"

    def _call_llm(self, messages: List[Dict[str, str]], temperature: float = 0) -> Dict[str, Any]:
        """Make a call to the LLM through the AI Proxy."""
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.ai_proxy_token}"
        }
        
        data = {
            "model": "gpt-4o-mini",  # AI Proxy only supports this model
            "messages": messages,
            "temperature": temperature
        }
        
        try:
            response = requests.post(self.ai_proxy_url, headers=headers, json=data)
            response.raise_for_status()
            
            # Log usage data
            cost = response.headers.get("cost", "N/A")
            monthly_cost = response.headers.get("monthlyCost", "N/A")
            monthly_requests = response.headers.get("monthlyRequests", "N/A")
            
            print(f"Request cost: ${cost}")
            print(f"Monthly usage: ${monthly_cost} ({monthly_requests} requests)")
            
            result = response.json()
            
            # Add to conversation history
            self.conversation_history.append({
                "timestamp": datetime.datetime.now().isoformat(),
                "request": data,
                "response": result,
                "usage_stats": {
                    "cost": cost,
                    "monthly_cost": monthly_cost,
                    "monthly_requests": monthly_requests
                }
            })
            
            # Save history periodically
            if len(self.conversation_history) % 5 == 0:
                self._save_history()
            
            return result
            
        except Exception as e:
            error_msg = f"Error calling LLM: {str(e)}"
            print(error_msg)
            return {"error": error_msg}

    def _save_history(self):
        """Save conversation history to a file."""
        try:
            with open(self.log_file, 'w') as f:
                json.dump(self.conversation_history, f, indent=2)
        except Exception as e:
            print(f"Error saving history: {str(e)}")

    def process(self, user_input: str, system_instructions: str = "You are a helpful assistant.") -> str:
        """Process a user input with standard instructions."""
        messages = [
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": user_input}
        ]
        
        result = self._call_llm(messages)
        
        if "error" in result:
            return f"Error: {result['error']}"
        
        return result["choices"][0]["message"]["content"]

    def process_with_reasoning(self, user_input: str, system_instructions: str = "You are a helpful assistant.") -> Dict[str, str]:
        """Process input and return both the response and reasoning."""
        reasoning_instructions = f"{system_instructions}\n\nWhen answering, first explain your reasoning step by step, then provide your final answer."
        
        messages = [
            {"role": "system", "content": reasoning_instructions},
            {"role": "user", "content": user_input}
        ]
        
        result = self._call_llm(messages)
        
        if "error" in result:
            return {"response": f"Error: {result['error']}", "reasoning": ""}
        
        full_response = result["choices"][0]["message"]["content"]
        
        # Try to separate reasoning from response
        try:
            # Simple parsing - can be improved for more structured outputs
            if "Reasoning:" in full_response and "Final Answer:" in full_response:
                reasoning, answer = full_response.split("Final Answer:")
                reasoning = reasoning.replace("Reasoning:", "").strip()
                answer = answer.strip()
            else:
                # Assume first 2/3 is reasoning, last 1/3 is answer
                split_point = len(full_response) * 2 // 3
                reasoning = full_response[:split_point].strip()
                answer = full_response[split_point:].strip()
                
            return {"response": answer, "reasoning": reasoning}
        except Exception:
            # If parsing fails, return everything
            return {"response": full_response, "reasoning": "Could not separate reasoning from response"}

    def chain_of_thought(self, user_input: str, system_instructions: str = "You are a helpful assistant.") -> Dict[str, str]:
        """Use explicit Chain of Thought prompting."""
        cot_prompt = f"""
        {system_instructions}
        
        I want you to solve this step-by-step:
        
        {user_input}
        
        Please follow this format in your response:
        
        Step 1: [First step in your reasoning]
        Step 2: [Second step in your reasoning]
        ...
        Final Answer: [Your final answer]
        """
        
        messages = [
            {"role": "system", "content": "You solve problems by showing your step-by-step reasoning before giving a final answer."},
            {"role": "user", "content": cot_prompt}
        ]
        
        result = self._call_llm(messages)
        
        if "error" in result:
            return {"steps": [], "answer": f"Error: {result['error']}"}
        
        full_response = result["choices"][0]["message"]["content"]
        
        # Parse the steps and final answer
        try:
            steps = []
            final_answer = ""
            
            lines = full_response.split('\n')
            for line in lines:
                line = line.strip()
                if line.startswith("Step"):
                    step_content = line.split(":", 1)[1].strip() if ":" in line else line
                    steps.append(step_content)
                elif line.startswith("Final Answer:"):
                    final_answer = line.replace("Final Answer:", "").strip()
            
            if not final_answer and lines:
                # If no explicit final answer, use the last line
                final_answer = lines[-1]
                
            return {"steps": steps, "answer": final_answer}
        except Exception as e:
            return {"steps": [], "answer": full_response, "parsing_error": str(e)}

    def explore_decision(self, user_input: str, system_instructions: str = "You are a helpful assistant.", temperatures: List[float] = [0.0, 0.5, 1.0]) -> List[Dict[str, Any]]:
        """Explore different possible decisions by varying temperature."""
        results = []
        
        base_messages = [
            {"role": "system", "content": f"{system_instructions}\nExplain your reasoning thoroughly before giving your final answer."},
            {"role": "user", "content": user_input}
        ]
        
        for temp in temperatures:
            print(f"Running with temperature {temp}...")
            result = self._call_llm(base_messages, temperature=temp)
            
            if "error" in result:
                response_text = f"Error: {result['error']}"
            else:
                response_text = result["choices"][0]["message"]["content"]
                
            results.append({
                "temperature": temp,
                "response": response_text
            })
            
            # Small delay to avoid rate limiting
            time.sleep(0.5)
            
        return results

    def competing_perspectives(self, user_input: str, system_instructions: str = "You are a helpful assistant.") -> Dict[str, str]:
        """Generate multiple perspectives on the same problem."""
        perspectives_prompt = f"""
        {system_instructions}
        
        Consider this problem: {user_input}
        
        Please analyze this from multiple perspectives:
        
        Perspective 1: [First approach or viewpoint]
        Reasoning: [Explain the reasoning for this perspective]
        
        Perspective 2: [Second approach or viewpoint]
        Reasoning: [Explain the reasoning for this perspective]
        
        Perspective 3: [Third approach or viewpoint, if applicable]
        Reasoning: [Explain the reasoning for this perspective]
        
        Analysis: [Compare the perspectives]
        
        Final Decision: [Your conclusion based on the analysis]
        """
        
        messages = [
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": perspectives_prompt}
        ]
        
        result = self._call_llm(messages)
        
        if "error" in result:
            return {"perspectives": [], "decision": f"Error: {result['error']}"}
        
        full_response = result["choices"][0]["message"]["content"]
        
        # Return the full structured response
        return {"full_analysis": full_response}

    def explicit_reasoning_trace(self, user_input: str, system_instructions: str = "You are a helpful assistant.") -> Dict[str, Any]:
        """Generate an explicit reasoning trace with factors and confidence."""
        trace_prompt = f"""
        {system_instructions}
        
        For the following request, please provide a detailed reasoning trace in this format:
        
        QUESTION: {user_input}
        
        FACTORS TO CONSIDER:
        - [List each relevant factor]
        
        REASONING TRACE:
        1. [First step in analysis]
        2. [Second step in analysis]
        3. [Additional steps as needed]
        
        UNCERTAINTIES:
        - [Any areas of uncertainty]
        
        CONFIDENCE: [Rate your confidence from 1-10]
        
        FINAL ANSWER: [Your conclusion]
        """
        
        messages = [
            {"role": "system", "content": "You provide detailed reasoning traces for complex questions."},
            {"role": "user", "content": trace_prompt}
        ]
        
        result = self._call_llm(messages)
        
        if "error" in result:
            return {"error": result["error"]}
        
        return {"full_trace": result["choices"][0]["message"]["content"]}

# Example usage
if __name__ == "__main__":
    # Initialize the LLM client
    llm_client = LLMClient()

    # Interactive CLI
    print("LLM Reasoning Explorer (type 'exit' to quit, 'debug' for debugging options)")
    print("Available modes: normal, reasoning, chain, explore, perspectives, trace")
    
    mode = "normal"
    system_instructions = "You are a helpful, accurate, and thoughtful assistant."
    
    while True:
        command = input("\nMode [" + mode + "] > ")
        
        if command.lower() == 'exit':
            break
        elif command.lower() == 'debug':
            print("\nDebug Options:")
            print("1. Change mode")
            print("2. Set system instructions")
            print("3. View conversation history")
            
            debug_choice = input("Select option: ")
            if debug_choice == '1':
                print("\nAvailable modes:")
                print("- normal: Standard response")
                print("- reasoning: Response with reasoning explanation")
                print("- chain: Chain of thought reasoning")
                print("- explore: Explore different temperatures")
                print("- perspectives: Multiple competing perspectives")
                print("- trace: Explicit reasoning trace with confidence")
                
                new_mode = input("Enter mode: ").lower()
                if new_mode in ['normal', 'reasoning', 'chain', 'explore', 'perspectives', 'trace']:
                    mode = new_mode
                else:
                    print("Invalid mode. Staying with current mode.")
            elif debug_choice == '2':
                print(f"\nCurrent system instructions: {system_instructions}")
                new_instructions = input("Enter new system instructions (or press enter to keep current): ")
                if new_instructions:
                    system_instructions = new_instructions
            elif debug_choice == '3':
                print(f"\nConversation history ({len(llm_client.conversation_history)} entries):")
                for i, entry in enumerate(llm_client.conversation_history[-3:]):
                    print(f"Entry {i+1} - {entry['timestamp']}")
                    print(f"Request: {entry['request']['messages'][-1]['content'][:50]}...")
                    if 'response' in entry and 'choices' in entry['response']:
                        print(f"Response: {entry['response']['choices'][0]['message']['content'][:50]}...")
                    print(f"Cost: {entry['usage_stats']['cost']}")
                    print("-" * 40)
        else:
            # Process the user query based on selected mode
            if mode == "normal":
                response = llm_client.process(command, system_instructions)
                print("\nResponse:", response)
            
            elif mode == "reasoning":
                result = llm_client.process_with_reasoning(command, system_instructions)
                print("\n=== REASONING ===")
                print(result["reasoning"])
                print("\n=== FINAL RESPONSE ===")
                print(result["response"])
            
            elif mode == "chain":
                result = llm_client.chain_of_thought(command, system_instructions)
                print("\n=== REASONING STEPS ===")
                for i, step in enumerate(result["steps"]):
                    print(f"Step {i+1}: {step}")
                print("\n=== FINAL ANSWER ===")
                print(result["answer"])
            
            elif mode == "explore":
                results = llm_client.explore_decision(command, system_instructions, temperatures=[0.0, 0.7, 1.0])
                for i, result in enumerate(results):
                    print(f"\n=== TEMPERATURE {result['temperature']} ===")
                    print(result["response"])
            
            elif mode == "perspectives":
                result = llm_client.competing_perspectives(command, system_instructions)
                print("\n=== MULTIPLE PERSPECTIVES ANALYSIS ===")
                print(result["full_analysis"])
            
            elif mode == "trace":
                result = llm_client.explicit_reasoning_trace(command, system_instructions)
                print("\n=== REASONING TRACE ===")
                print(result["full_trace"])

LLM Reasoning Explorer (type 'exit' to quit, 'debug' for debugging options)
Available modes: normal, reasoning, chain, explore, perspectives, trace
Request cost: $N/A
Monthly usage: $N/A (N/A requests)

Response: Reasoning is the cognitive process of drawing conclusions, making inferences, or forming judgments based on available information or evidence. It involves the ability to think logically, analyze situations, and solve problems. There are several types of reasoning, including:

1. **Deductive Reasoning**: This involves starting with a general statement or hypothesis and examining the possibilities to reach a specific, logical conclusion. For example, if all humans are mortal (general statement) and Socrates is a human, then Socrates is mortal (specific conclusion).

2. **Inductive Reasoning**: This type of reasoning involves looking at specific instances or observations and making broader generalizations. For example, if you observe that the sun has risen in the east every day o

The kernel is the central component of Semantic Kernel. At its simplest, the kernel is a Dependency Injection container that manages all of the services and plugins necessary to run your AI application. If you provide all of your services and plugins to the kernel, they will then be seamlessly used by the AI as needed.

In [16]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.ollama import OllamaChatPromptExecutionSettings


In [3]:
chat_completion_service = OllamaChatCompletion(
    ai_model_id="gemma3:4b",
    service_id="gemma3", # Optional; for targeting specific services within Semantic Kernel
)

In [5]:
# Add the chat completion service created above to the kernel
kernel=Kernel()
kernel.add_service(chat_completion_service)


In [7]:
# Retrieve the chat completion service by id
chat_completion_service = kernel.get_service(service_id="gemma3")

In [15]:
#streaming chat application

chat_history = ChatHistory()
chat_history.add_user_message("Hello, how are you?")

response = chat_completion_service.get_streaming_chat_message_content(
    chat_history=chat_history,
    settings=OllamaChatPromptExecutionSettings()
)

async for chunk in response:
    print(chunk, end="")

I’m doing well, thank you for asking! As a large language model, I don’t really *feel* in the way humans do, but my systems are running smoothly and I’m ready to chat. 😊 

How are *you* doing today? Is there anything you’d like to talk about, or maybe I can help you with something?

main agents using agno

In [13]:
from agno.agent import Agent
from agno.models.ollama import Ollama
from agno.tools.googlesearch import GoogleSearchTools

In [ ]:
#while running an agent dont forget to give description and instructions to it a sample webscrapping agent
agent = Agent(
    model=Ollama("nemotron-mini:4b"),
    tools=[
        GoogleSearchTools(),
    ],
    description="you are an helpful web scrapper who helps me scrape data from any website.",
    instructions=[
        "Given a link by the user, respond with the top 10 highlighted features scrapped from the website"
        ".",
    ],
    show_tool_calls=True
)
response=agent.run("https://getstream.io/blog/multiagent-ai-frameworks/")
print(response.content)


 Top 10 features of GetStream MultiAgent AI Frameworks:
1. Extensible and customizable architecture
2. Open-source libraries and tools
3. Strong support for various programming languages
4. Extensive documentation and community support
5. Integration with popular frameworks like Spring, React, etc.
6. Scalability to handle large datasets and complex simulations
7. Advanced machine learning capabilities
8. Real-time data processing and analysis
9. Data privacy and security measures in place
10. Comprehensive set of features for multiagent system development


In [ ]:
from agno.agent import Agent
from agno.models.ollama import Ollama
from agno.tools.googlesearch import GoogleSearchTools
from agno.tools.yfinance import YFinanceTools
from agno.tools.website import WebsiteTools
import asyncio
import nest_asyncio  # For web scraping

nest_asyncio.apply()  # Apply the patch to allow nested event loops

# Web scraping agent
web_agent = Agent(
    name="Web Scraper",
    role="Scrape data from websites",
    model=Ollama("nemotron-mini:4b"),
    tools=[
        WebsiteTools(),  # Use WebTools for scraping instead of GoogleSearchTools
        GoogleSearchTools()  # Keep this for searching
    ],
    description="You are a helpful web scraper who extracts data from websites.",
    instructions=[
        "Given a link by the user, extract the relevant information from the website.",
        "Use the web scraping tool to get the content of web pages.",
        "Use search when you need to find relevant websites first."
    ],
    show_tool_calls=True,
    markdown=True
)

# Finance agent
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=Ollama("nemotron-mini:4b"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    description="You are a finance expert who provides financial data and analysis.",
    instructions=[
        "Use tables to display financial data.",
        "Provide clear explanations of financial metrics.",
        "Always cite the source of your financial data."
    ],
    show_tool_calls=True,
    markdown=True
)

# Team agent
agent_team = Agent(
    team=[web_agent, finance_agent],
    model=Ollama("nemotron-mini:4b"),
    description="You are a research assistant that combines web scraping and financial analysis capabilities.",
    instructions=[
        "Use the Finance Agent to get financial data from yfinance.",
        "Use the Web Scraper to find and extract relevant information from websites.",
        "Always include sources for your information.",
        "Use tables to display structured data.",
        "Provide a comprehensive analysis that combines financial data with market insights."
    ],
    show_tool_calls=True,
    markdown=True,debug_mode=True
)


In [8]:
from autodistill_metaclip import MetaCLIP
from autodistill.detection import CaptionOntology
import os

# Define an ontology to map class names to our MetaCLIP prompt
# the ontology dictionary has the format {caption: class}
# where caption is the prompt sent to the base model, and class is the label that will
# be saved for that caption in the generated annotations
# then, load the model
base_model = MetaCLIP(
    ontology=CaptionOntology(
        {
            "person": "person",
            "a forklift": "forklift"
        }
    )
)

# Specify the correct pretrained tag
pretrained_tag = "laion400m_e31"  # Use a valid pretrained tag

# Load the model with the correct pretrained tag
base_model = MetaCLIP(
    ontology=CaptionOntology(
        {
            "person": "person",
            "a forklift": "forklift"
        }
    )
)

results = base_model.predict("C:\AI_PROJECTS\Vitstuff\WhatsApp Image 2025-03-07 at 23.26.04_1e48c753.jpg")
print(results)

invalid escape sequence '\A'
invalid escape sequence '\A'
invalid escape sequence '\A'
invalid escape sequence '\A'


RuntimeError: Pretrained weights (C:\Users\rexjo/.cache/autodistill/open_clip/b32_400m.pt) not found for model ViT-B-32-quickgelu. Available pretrained tags (['openai', 'laion400m_e31', 'laion400m_e32', 'metaclip_400m', 'metaclip_fullcc'].